In [1]:
"""
Glen Williams
Meta Take Home
BSAD 8630
Quick note, I chose to do this using python rather than excel becuase that is what I use at work, 
and I could use the practice
"""

#import statements
import pandas as pd
import numpy as np
import requests as re
#importing Yahoo Finance API
import yfinance as yf
from yfinance.utils import auto_adjust
import plotly.express as px

#Define tickers
tickers = {"^RUT": "^RUT",
            "META": "META"}
#Date range
start = "2021-10-01"
end = "2025-09-30"

#Get data from Yahoo Finance
data = yf.download(list(tickers.values()), start = start, end = end, interval = "1mo", auto_adjust = True, progress = True)

#Split the data into two dataframes
rut_df = data.xs(tickers["^RUT"], axis=1, level=1)
meta_df = data.xs(tickers["META"], axis=1, level=1)

# Clean up index
rut_df.index = pd.to_datetime(rut_df.index)
meta_df.index = pd.to_datetime(meta_df.index)

#keep only closing prices
rut_df  = rut_df[["Close"]]
meta_df = meta_df[["Close"]]

#add Monthly Return
rut_df["Return"] = rut_df["Close"].pct_change()*100
meta_df["Return"] = meta_df["Close"].pct_change()*100

#Preview of dataframes
print("^RUT data:")
print(rut_df, "\n")

print("META data:")
print(meta_df)


[*********************100%***********************]  2 of 2 completed

^RUT data:
Price             Close     Return
Date                              
2021-10-01  2297.189941        NaN
2021-11-01  2198.909912  -4.278272
2021-12-01  2245.310059   2.110143
2022-01-01  2028.449951  -9.658359
2022-02-01  2048.090088   0.968234
2022-03-01  2070.129883   1.076115
2022-04-01  1864.099976  -9.952511
2022-05-01  1864.040039  -0.003215
2022-06-01  1707.989990  -8.371604
2022-07-01  1885.229980  10.377109
2022-08-01  1844.119995  -2.180635
2022-09-01  1664.719971  -9.728219
2022-10-01  1846.859985  10.941180
2022-11-01  1886.579956   2.150676
2022-12-01  1761.250000  -6.643236
2023-01-01  1931.939941   9.691409
2023-02-01  1896.989990  -1.809060
2023-03-01  1802.479980  -4.982104
2023-04-01  1768.989990  -1.857995
2023-05-01  1749.650024  -1.093277
2023-06-01  1888.729980   7.949016
2023-07-01  2003.180054   6.059631
2023-08-01  1899.680054  -5.166785
2023-09-01  1785.099976  -6.031546
2023-10-01  1662.280029  -6.880284
2023-11-01  1809.020020   8.827634
2023-12-0

In [2]:
"""Part 1: Average return and standard deviation"""

#Calculate average return
rut_average_return = rut_df["Return"].mean()
meta_average_return = meta_df["Return"].mean()

print(f'The average monthly return of ^RUT is {round(rut_average_return, 2)}%')
print(f'The average monthly return of META is {round(meta_average_return, 2)}%')

#Calculate standard deviation
rut_std_dev = rut_df["Return"].std()
meta_std_dev = meta_df["Return"].std()
print(f"The standard deviation of ^RUT's monthly returns is {round(rut_std_dev, 4)}%")
print(f"The standard deviation of META's monthly returns is {round(meta_std_dev, 4)}%")

The average monthly return of ^RUT is 0.32%
The average monthly return of META is 2.61%
The standard deviation of ^RUT's monthly returns is 6.3455%
The standard deviation of META's monthly returns is 12.8162%


In [3]:
"""Part 2: Finding the value of a news event"""
# Merge monthly returns
returns_df = pd.merge(
    rut_df["Return"],
    meta_df["Return"],
    left_index=True,
    right_index=True,
    suffixes=("_RUT", "_META")).dropna()

# Regression: META ~ RUT
X = returns_df["Return_RUT"]
y = returns_df["Return_META"]

beta, alpha = np.polyfit(X, y, 1)

# R-squared
corr = np.corrcoef(X, y)[0, 1]
r_squared = corr ** 2

# Scatter plot with regression line
fig = px.scatter(
    returns_df,
    x="Return_RUT",
    y="Return_META",
    trendline="ols",
    title="META vs Russell 2000 Monthly Returns")
fig.show()

print(f"Beta (slope): {beta:.4f}")
print(f"R-squared: {r_squared:.4f}")

Beta (slope): 0.5053
R-squared: 0.0626


In [7]:
# Daily prices for the event window
event_start = "2025-10-28"
event_end   = "2025-10-31"

daily_prices = yf.download(
    ["^RUT", "META"],
    start=event_start,
    end=event_end,
    interval="1d",
    auto_adjust=True,
    progress=False
)["Close"]

print(daily_prices, '\n')

# One-day returns (Oct 29 -> Oct 30)
daily_returns = daily_prices.pct_change().loc["2025-10-30"] * 100

rut_1d_return  = daily_returns["^RUT"]
meta_1d_return = daily_returns["META"]

print(f"Russell 2000 1-day return: {rut_1d_return:.4f}%")
print(f"META 1-day return: {meta_1d_return:.4f}%")

Rf = 0
Rm = rut_1d_return

meta_capm_return = Rf + beta * (Rm - Rf)

print(f"CAPM expected META return: {meta_capm_return:.4f}%")

abnormal_return = meta_1d_return - meta_capm_return

print(f"Actual META 1-day return: {meta_1d_return:.4f}%")
print(f"Abnormal (news-driven) return: {abnormal_return:.4f}%" '\n')
print("""Meta's actual return for the trading period of interest was far larger than the market move, 
well outside what CAPM would attribute to systematic risk. 
By my calcuations, approximately 10.95 percent of of Meta's performance over this period can be attributed to events within that period.""")


Ticker            META         ^RUT
Date                               
2025-10-28  750.827637  2506.649902
2025-10-29  751.057434  2484.810059
2025-10-30  665.926880  2465.949951 

Russell 2000 1-day return: -0.7590%
META 1-day return: -11.3348%
CAPM expected META return: -0.3835%
Actual META 1-day return: -11.3348%
Abnormal (news-driven) return: -10.9512%

Meta's actual return for the trading period of interest was far larger than the market move, 
well outside what CAPM would attribute to systematic risk. 
By my calcuations, approximately 10.95 percent of of Meta's performance over this period can be attributed to events within that period.
